In [63]:
import pandas as pd
import requests
import zkey
from citipy import citipy

import sqlalchemy
from sqlalchemy import create_engine, func

from sqlalchemy import create_engine

In [64]:
## Database Setup

In [65]:
engine = create_engine("sqlite:///etl.sqlite")

## Yelp Business Data

In [66]:
y_bus = pd.read_csv("Resources/yelp_academic_dataset_business_copy.csv")

In [67]:
y_bus.head()

,business_id,categories,longitude,latitude,review_count,city,stars
0,vcNAWiLM4dR7D2nwwJ7nCA,Doctors;Health & Medical,-111.983758,33.499313,7,Phoenix,3.5
1,JwUE5GmEO-sH1FuwJgKBlQ,Restaurants,-89.335844,43.238893,26,De Forest,4.0
2,uGykseHzyS5xAMWoN6YUqA,American (Traditional);Restaurants,-89.353437,43.252267,16,De Forest,4.0
3,LRKJF43s9-3jG9Lgx4zODg,Food;Ice Cream & Frozen Yogurt;Fast Food;Resta...,-89.374983,43.251045,7,De Forest,4.5
4,RgDg-k9S5YD_BaxMckifkg,Chinese;Restaurants,-89.343722,43.240875,3,De Forest,4.0


In [68]:
print(len(y_bus))

42153


In [69]:
yelp_business = pd.DataFrame(y_bus)
yelp_business.head()

,business_id,categories,longitude,latitude,review_count,city,stars
0,vcNAWiLM4dR7D2nwwJ7nCA,Doctors;Health & Medical,-111.983758,33.499313,7,Phoenix,3.5
1,JwUE5GmEO-sH1FuwJgKBlQ,Restaurants,-89.335844,43.238893,26,De Forest,4.0
2,uGykseHzyS5xAMWoN6YUqA,American (Traditional);Restaurants,-89.353437,43.252267,16,De Forest,4.0
3,LRKJF43s9-3jG9Lgx4zODg,Food;Ice Cream & Frozen Yogurt;Fast Food;Resta...,-89.374983,43.251045,7,De Forest,4.5
4,RgDg-k9S5YD_BaxMckifkg,Chinese;Restaurants,-89.343722,43.240875,3,De Forest,4.0


In [70]:
type(yelp_business)

pandas.core.frame.DataFrame

In [71]:
yelp_business.isna().sum()

business_id       0
categories      339
longitude         0
latitude          0
review_count      0
city              0
stars             0
dtype: int64

In [72]:
yelp_business = yelp_business.dropna(how='any')

In [73]:
yelp_business.isna().sum()

business_id     0
categories      0
longitude       0
latitude        0
review_count    0
city            0
stars           0
dtype: int64

In [74]:
yelp_business.duplicated().sum()

0

In [75]:
yelp_business.describe()

,longitude,latitude,review_count,stars
count,41814.000000,41814.000000,41814.000000,41814.000000
mean,-104.051942,36.534810,29.530014,3.677166
std,28.430954,5.835889,92.844505,0.884427
min,-115.369725,32.876638,3.000000,1.000000
25%,-115.136178,33.476500,4.000000,3.000000
50%,-112.067534,33.678615,8.000000,3.500000
75%,-111.882191,36.142010,22.000000,4.500000
max,-3.050088,56.033777,4084.000000,5.000000


### Yelp Reviews

In [76]:
y_rev = pd.read_csv("Resources/yelp_academic_dataset_review.copy.csv")

In [77]:
y_rev.head()

,text,votes.cool,business_id,votes.funny,stars,date,type,votes.useful
0,dr. goldberg offers everything i look for in a...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,5,5/17/2007,review,2
1,"Unfortunately, the frustration of being Dr. Go...",0,vcNAWiLM4dR7D2nwwJ7nCA,0,2,3/22/2010,review,2
2,Dr. Goldberg has been my doctor for years and ...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,4,2/14/2012,review,1
3,Been going to Dr. Goldberg for over 10 years. ...,0,vcNAWiLM4dR7D2nwwJ7nCA,0,4,3/2/2012,review,0
4,Got a letter in the mail last week that said D...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,4,5/15/2012,review,2


In [78]:
print(len(y_rev))

1048575


In [79]:
yelp_reviews = pd.DataFrame(y_rev)
yelp_reviews.head()

,text,votes.cool,business_id,votes.funny,stars,date,type,votes.useful
0,dr. goldberg offers everything i look for in a...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,5,5/17/2007,review,2
1,"Unfortunately, the frustration of being Dr. Go...",0,vcNAWiLM4dR7D2nwwJ7nCA,0,2,3/22/2010,review,2
2,Dr. Goldberg has been my doctor for years and ...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,4,2/14/2012,review,1
3,Been going to Dr. Goldberg for over 10 years. ...,0,vcNAWiLM4dR7D2nwwJ7nCA,0,4,3/2/2012,review,0
4,Got a letter in the mail last week that said D...,1,vcNAWiLM4dR7D2nwwJ7nCA,0,4,5/15/2012,review,2


In [80]:
type(yelp_reviews)

pandas.core.frame.DataFrame

In [81]:
yelp_reviews.isna().sum()

text            1
votes.cool      0
business_id     0
votes.funny     0
stars           0
date            0
type            0
votes.useful    0
dtype: int64

In [82]:
yelp_reviews = yelp_reviews.dropna(how = 'any')

In [83]:
yelp_reviews.isna().sum()

text            0
votes.cool      0
business_id     0
votes.funny     0
stars           0
date            0
type            0
votes.useful    0
dtype: int64

In [84]:
yelp_reviews.duplicated().sum()

2

In [85]:
yelp_reviews = yelp_reviews.drop_duplicates()

In [86]:
yelp_reviews.duplicated().sum()

0

In [87]:
fruits = ["apple", "banana", "cherry"]

for i in fruits

SyntaxError: invalid syntax (<ipython-input-87-20264d66f784>, line 3)

In [ ]:
### Merging dataframe

In [ ]:
total_yelp = pd.merge(y_bus,y_rev, how="outer", on="business_id")

total_yelp.head()

In [ ]:
total_yelp=total_yelp.sample(n=4000, replace='False')

len(total_yelp)

In [ ]:
total_yelp.isna().sum()

In [ ]:
total_yelp = total_yelp.dropna(how='any')

In [ ]:
total_yelp.isna().sum()

In [ ]:
total_yelp.duplicated().sum()

In [ ]:
total_yelp = total_yelp.drop_duplicates()

In [ ]:
total_yelp.to_sql(yelp.data)

## Zomato API

In [ ]:
url = "https://developers.zomato.com/api/v2.1/search?"



In [ ]:
header = {"Accept" : "application/json", "user_key": "0092256f05a628b53994aec9ae1f30e0"}

response = requests.get(url, headers=header).json()

print(url)

In [ ]:
print(response)

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
   city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

   # If the city is unique, then add it to a our cities list
   if city not in cities:
       cities.append(city)

# Print the city count to confirm sufficient count
total = len(cities)

In [ ]:
query_url = “https://developers.zomato.com/api/v2.1/cities?q=”
city_num = 0
api_data = open(‘output_data/locations.csv’, ‘w’)
csvwriter = csv.writer(api_data)
header = [‘Location ID’]
csvwriter.writerow(header)

for city in cities:
   url_header = {“Accept”: “application/json”, “user_key”: “4ae5aec1f3577713b3ece44fc9c3d123"}
   response = requests.get(query_url + city, headers=url_header).json()
   city_num += 1
   try:
       csvwriter.writerow([response[‘id’]])
       print(f”Processing Record {city_num} of {total} | {city}“)
   except:
       print(f”City not found. Skipping...“)
       continue
api_data.close()

In [ ]:
query_url = “https://developers.zomato.com/api/v2.1/search?entity_id=”
city_num = 0
api_data = open(‘output_data/zomato.csv’, ‘w’)
csvwriter = csv.writer(api_data)
header = [‘City’, ‘Restaurant’, ‘Lat’, ‘Lng’, ‘Rating’, ‘Rating Text’]
csvwriter.writerow(header)

for city in cities:
   url_header = {“Accept”: “application/json”, “user_key”: “0092256f05a628b53994aec9ae1f30e0"}
   response = requests.get(query_url + city, headers=url_header).json()
   city_num += 1
   try:
       csvwriter.writerow([response[‘location’][‘city’], response[‘name’], response[‘location’][‘latitude’],
                   response[‘location’][‘longitude’], response[‘user_rating’][‘aggregate_rating’], response[‘user_rating’][‘rating_text’]])
       print(f”Processing Record {city_num} of {total} | {city}“)
   except:
       print(f”City not found. Skipping...“)
       continue
api_data.close()
Message Input


Message Danny, shahzina

In [ ]:
url1 = "https://developers.zomato.com/api/v2.1/geocode?"

query_url1 = url1 + "user-key=" + zkey

response1 = requests.get(query_url1, headers={'user-key': zkey}).json()
pprint(response1)

In [ ]:
city_list = []

for i in response1:
    city= response1['location']['city_name']
    
    city_list.append(city)

In [ ]:
url2 = "https://developers.zomato.com/api/v2.1/search?"
query_url2 = url2 + "user-key=" + zkey

response2 = requests.get(query_url2, headers={'user-key': zkey}).json()
pprint(response2)

In [ ]:
data = {'city' :[], 'latitude': [], 'longitude':[],'review' :[], 'rating': [], 'rating_text':[] }
df = pd.DataFrame(data)
df.head()

In [ ]:
city_list = []
lat_list = []
long_list = []
review_list = []
rating_list = []
ratingtext_list = []

for i in response2:
    rest = response2['restaurants']
    city = rest['restaurant'] #['city']
    
    city_list.append(city)

In [ ]:
city_list